In [10]:
import xml.etree.ElementTree as ET

Parse xml file for white matter regions and store both the index and the name of the region

In [11]:
file = '/usr/local/fsl/data/atlases/Talairach.xml' #From FSL default installation
tree = ET.parse(file)
root = tree.getroot()

In [12]:
white_matter_indices = []
associated_labels = []
for child in root[1]:
    #print(child.attrib['index'], child.text)
    if "White Matter" not in child.text:
        continue
    else: 
        white_matter_indices.append(child.attrib['index'])
        associated_labels.append(child.text)

In [13]:
len(associated_labels)

194

In [14]:
split_a_lab_test = associated_labels[0].split(".")
limited_string_test = split_a_lab_test[0] + "." + split_a_lab_test[1] + "." + split_a_lab_test[2]
limited_string_test
#for word in split_a_lab_test[0:3]:
#    print(word)

'Left Cerebrum.Temporal Lobe.Inferior Temporal Gyrus'

In [15]:
#import os #Only needed if subjectwise average FA in Talairach atlas space needs to be computed and saved
#import sys
import pandas as pd
import numpy as np

import nibabel as nib
#from nibabel import processing as prc #only needed if resampling of subject data to atlas space is necessary

import scipy as sp
import scipy.stats as stats
import statsmodels.stats.multitest as mt

import matplotlib as mpl
import matplotlib.pyplot as plt

with open('/media/corey/4TB-WDBlue/data-thesis/DTI/Talairach_WM_regions_list.txt', 'a') as file:
    for v, label in enumerate(white_matter_indices):
        file.write('volume_%s.nii.gz\n' %label) 

with open('/media/corey/4TB-WDBlue/data-thesis/DTI/Talairach_WM_regions_list2.txt', 'a') as file:
    for v, label in enumerate(white_matter_indices):
        print(label)
        if os.path.isfile('/media/corey/2_4TB-WDBlue/DTI_merged_data/1006/4o/averageFAs/Talairach/av_%s_vol.nii.gz' %label) == False:
            continue
        file.write('av_%s_vol.nii.gz\n' %label) 

In [16]:
subjectDir = str('/media/corey/2_4TB-WDBlue/DTI_merged_data/')
DirExtName = str('/4o/averageFAs/Talairach')
FA_filenames = np.loadtxt('/media/corey/4TB-WDBlue/data-thesis/DTI/Talairach_WM_regions_list.txt',dtype=str)
#Load subjectlist of interest, store IDs as integers
sub_list = np.loadtxt('/media/corey/4TB-WDBlue/data-thesis/subsWithDTI.txt',dtype=str)
#Store subject names as a list with a standard suffix if provided
sub_names = sub_list
subjectInfo = pd.read_csv(open('/media/corey/4TB-WDBlue/data-thesis/subject_info.csv'),delim_whitespace=True,usecols=(4,11))

In [17]:
subject_list = open('/media/corey/4TB-WDBlue/data-thesis/subject_list_full.txt')
sublist = subject_list.readlines()
ext_subjectDir = '/media/corey/2_4TB-WDBlue/ext_data/EP_FunctionalPreprocessing'
DTI_dir = '/media/corey/2_4TB-WDBlue/DTI_merged_data'

In [18]:
FA_test_file = nib.load('/media/corey/2_4TB-WDBlue/DTI_merged_data/1009/4o/FLIRTed_meanfsumsamples.nii.gz')
FA_test_data = FA_test_file.get_fdata()
print(FA_test_data.shape)
FA_test_affine = FA_test_file.affine

(91, 109, 91)


In [19]:
Talairach_file = '/usr/local/fsl/data/atlases/Talairach/Talairach-labels-2mm.nii.gz'
Talairach_data = nib.load(Talairach_file)
T_data = Talairach_data.get_fdata()

#Try again using individual regions in isolated images
for v, label in enumerate(white_matter_indices):
    current_volume = np.zeros_like(T_data)
    current_value = int(label)
    for x in range(0,T_data.shape[0]):
        for y in range(0, T_data.shape[1]):
            for z in range(0,T_data.shape[2]):
                if T_data[x,y,z] == current_value:
                    current_volume[x,y,z] = 1
    current_image = nib.Nifti1Image(current_volume, Talairach_data.affine)
    nib.save(current_image,'/home/corey/atlases/Talairach/individuals/volume_%s.nii.gz' %label)

In [20]:
#resampled_current = prc.resample_from_to(current_image,FA_test_file,order=0)
#    resampled_c_o1 = prc.resample_from_to(current_image,FA_test_file,order=1)
#    resampled_data_o1 = resampled_c_o1.get_fdata()
#    for x in range(0,resampled_data_o1.shape[0]):
#        for y in range(0, resampled_data_o1.shape[1]):
#            for z in range(0,resampled_data_o1.shape[2]):
#                if resampled_data_o1[x,y,z] != 0:
##                    resampled_data_o1[x,y,z] = 1
 #   binarized_resampled_o1_image = nib.Nifti1Image(resampled_data_o1, FA_test_affine)
    #nib.save(resampled_current,'/home/corey/atlases/Talairach/resampled_individuals/volume_%s.nii.gz' %label)
    #nib.save(resampled_c_o1,'/home/corey/atlases/Talairach/resampled_indiv_order1/volume_%s.nii.gz' %label)
#    nib.save(binarized_resampled_o1_image,'/home/corey/atlases/Talairach/binarized_o1s/volume_%s.nii.gz' %label)

In [21]:
Atlas_path = '/home/corey/atlases/Talairach/individuals/'

#convolve atlas images with data from each subject, then save to each subject's directory
for n in sub_names:
    print(n)
    subj_FA_data = nib.load(subjectDir + n + '/4o/FLIRTed_meanfsumsamples.nii.gz')
    s_data = subj_FA_data.get_fdata()
    for i, v in enumerate(FA_filenames):
        atlas_template = Atlas_path + v
        template_volume = nib.load(atlas_template)
        t_data = template_volume.get_fdata()
        #count_temp_nnz = np.count_nonzero(t_data)
        #print(count_temp_nnz)
        convolved_data = np.zeros_like(FA_test_data)
        av_c_data = np.zeros_like(FA_test_data)
        for x in range(0,t_data.shape[0]):
            for y in range(0,t_data.shape[1]):
                for z in range(0,t_data.shape[2]):
                    if t_data[x,y,z] == 1:
                        convolved_data[x,y,z] = s_data[x,y,z]
        convolved_image = nib.Nifti1Image(convolved_data, FA_test_affine)
        nib.save(convolved_image,subjectDir + n + '/4o/averageFAs/%s_vol.nii.gz' %white_matter_indices[i])
        num_nonzero_voxels = np.count_nonzero(convolved_data)
        if num_nonzero_voxels <= 10:
            print(v, 'not enough non-zero voxels')
            continue
        total_FA = np.sum(convolved_data)
        average_value =  total_FA / num_nonzero_voxels  # np.mean(convolved_data)
        for x in range(0,t_data.shape[0]):
            for y in range(0,t_data.shape[1]):
                for z in range(0,t_data.shape[2]):
                    if convolved_data[x,y,z] != 0:
                        av_c_data[x,y,z] = average_value
        av_c_im = nib.Nifti1Image(av_c_data, FA_test_affine)
        nib.save(av_c_im, subjectDir + n + '/4o/averageFAs/Talairach/av_%s_vol.nii.gz' %white_matter_indices[i])

In [22]:
T_data.shape

(91, 109, 91)

#Try again using individual regions in isolated images
for v, label in enumerate(white_matter_indices):
    current_volume = np.zeros_like(T_data)
    current_value = int(label)
    for x in range(0,T_data.shape[0]):
        for y in range(0, T_data.shape[1]):
            for z in range(0,T_data.shape[2]):
                if T_data[x,y,z] == current_value:
                    current_volume[x,y,z] = 1
    current_image = nib.Nifti1Image(current_volume, Talairach_data.affine)
    #nib.save(current_image,'/home/corey/atlases/Talairach/individuals/volume_%s.nii.gz' %label)
    resampled_current = prc.resample_from_to(current_image,FA_test_file,order=0)
    resampled_c_o1 = prc.resample_from_to(current_image,FA_test_file,order=1)
    resampled_data_o1 = resampled_c_o1.get_fdata()
    for x in range(0,resampled_data_o1.shape[0]):
        for y in range(0, resampled_data_o1.shape[1]):
            for z in range(0,resampled_data_o1.shape[2]):
                if resampled_data_o1[x,y,z] != 0:
                    resampled_data_o1[x,y,z] = 1
    binarized_resampled_o1_image = nib.Nifti1Image(resampled_data_o1, FA_test_affine)
    #nib.save(resampled_current,'/home/corey/atlases/Talairach/resampled_individuals/volume_%s.nii.gz' %label)
    #nib.save(resampled_c_o1,'/home/corey/atlases/Talairach/resampled_indiv_order1/volume_%s.nii.gz' %label)
    nib.save(binarized_resampled_o1_image,'/home/corey/atlases/Talairach/binarized_o1s/volume_%s.nii.gz' %label)

In [23]:
import os

In [24]:
test_path = subjectDir + '1006' + DirExtName + '/av_14_vol.nii.gz'

In [25]:
os.path.isfile(test_path)

True

In [26]:
control_FA = []
nonaff_FA = []

n_control = 0
n_nonaff = 0
for n in sub_names:
    path_to_data = subjectDir + n + DirExtName
    phen = subjectInfo.loc[subjectInfo['src_subject_id'] == n]
    pdesc = phen['phenotype_description'].values[0]
    for r in white_matter_indices:

        fa_filename = path_to_data + '/av_%s_vol.nii.gz' %r
        if os.path.isfile(fa_filename) == False:
            continue
        fa_file = nib.load(fa_filename)
        fa_data = fa_file.get_fdata()
        FA_val = np.max(fa_data)
        if pdesc == 'In,good,health':
            control_FA.append([FA_val, r])
            
        if pdesc == 'Non-affective,psychosis':
            nonaff_FA.append([FA_val, r])
    if pdesc == 'In,good,health':
        n_control += 1
    if pdesc == 'Non-affective,psychosis':
        n_nonaff +=1


In [27]:
FA_filenames = np.loadtxt('/media/corey/4TB-WDBlue/data-thesis/DTI/Talairach_WM_regions_list2.txt',dtype=str)

In [28]:
subjectDir + '1006' + DirExtName + '/' + FA_filenames[0]

'/media/corey/2_4TB-WDBlue/DTI_merged_data/1006/4o/averageFAs/Talairach/av_14_vol.nii.gz'

In [29]:
associated_labels[0]

'Left Cerebrum.Temporal Lobe.Inferior Temporal Gyrus.White Matter.*'

In [30]:
white_matter_indices[1]

'20'

In [31]:
stripped_wm_list = []
stripped_associated_labels = []
for v_ind, v in enumerate(white_matter_indices):
    if os.path.isfile(subjectDir + '1006' + DirExtName + '/' + 'av_%s_vol.nii.gz' %v) == False:
        continue
    stripped_wm_list.append(v)
    split_label = associated_labels[v_ind].split(".")
    limited_label = split_label[0] + "." + split_label[1] + "." + split_label[2]
    stripped_associated_labels.append(limited_label)

In [32]:
c_rw_s = []
n_rw_s = []

compiled_c_fa = []
c_matrix = np.zeros([n_control, len(stripped_wm_list)])
n_matrix = np.zeros([n_nonaff, len(stripped_wm_list)])
compiled_n_fa = []

c_rw_std = []
n_rw_std = []
for v_ind, v in enumerate(stripped_wm_list):
    if os.path.isfile(subjectDir + '1006' + DirExtName + '/' + 'av_%s_vol.nii.gz' %v) == False:
        continue
    sum = 0
    count = 0
    for f_ind, f in enumerate(control_FA):
        if f[1] == v:
            sum += f[0]
            c_matrix[count, v_ind] = f[0]
            count += 1
    c_average_fa_v = sum / count
    fsum = 0
    fcount = 0
    for f_ind, f in enumerate(nonaff_FA):
        if f[1] == v:
            fsum += f[0]
            n_matrix[fcount, v_ind] = f[0]
            fcount += 1
    n_average_fa_v = fsum / fcount
    c_rw_s.append([c_average_fa_v, v])
    n_rw_s.append([n_average_fa_v, v])

    #Standard deviations
    sq_diff_from_average = 0
    n_sq_diff_from_average = 0
    for f_ind, f in enumerate(control_FA):
        if f[1] == v:
            sq_diff_from_average += np.square(f[0] - c_rw_s[v_ind][0])
    sd = np.sqrt(sq_diff_from_average / (count - 1))
    c_rw_std.append([sd,v])
    for f_ind, f in enumerate(nonaff_FA):
        if f[1] == v:
            n_sq_diff_from_average += np.square(f[0] - n_rw_s[v_ind][0])
    f_sd = np.sqrt(n_sq_diff_from_average / (fcount - 1))
    n_rw_std.append([f_sd,v])


In [33]:
#T-tests
p_vals_cn = []
pvals_2samp = []
diffs_cn = []
diff_cn_cis = []

calculated_cis = []
for v_ind, v in enumerate(stripped_wm_list):
    test_results = stats.ttest_ind_from_stats(c_rw_s[v_ind][0], c_rw_std[v_ind][0], count, n_rw_s[v_ind][0], n_rw_std[v_ind][0], fcount, alternative='greater')
    
    ttest_2samp = stats.ttest_ind(c_matrix[:,v_ind],n_matrix[:,v_ind], equal_var=False, alternative='greater')
    
    p_vals_cn.append(test_results[1])
    pvals_2samp.append(ttest_2samp[1])

    diff_means = c_rw_s[v_ind][0] - n_rw_s[v_ind][0]
    diffs_cn.append(diff_means)

    ci = ttest_2samp.confidence_interval(confidence_level=0.95)
    diff_cn_cis.append(ci)


In [34]:
fdr_corrected_cn = mt.fdrcorrection(p_vals_cn, alpha=0.05)

In [35]:
fdr_corrected_2samp = mt.fdrcorrection(pvals_2samp, alpha=0.05)

In [36]:
lower_bounds_95cis = []
for v_ind, v in enumerate(diff_cn_cis):
    lower_bounds_95cis.append(diff_cn_cis[v_ind][0])

In [37]:
differences_fdr_corrected_table = pd.DataFrame(columns=['Label ID','Region Name','\u0394','FDR P-val', '95% CI', 'P-val'])
differences_fdr_corrected_table['Label ID'] = stripped_wm_list #Talairach Atlas Label ID
differences_fdr_corrected_table['Region Name'] = stripped_associated_labels #First column is region name
differences_fdr_corrected_table['\u0394'] = diffs_cn #Second column is absolute difference in FA
differences_fdr_corrected_table['FDR P-val'] = fdr_corrected_2samp[1] #Third column is corrected pval
differences_fdr_corrected_table['95% CI'] = lower_bounds_95cis #Fourth column is one sided confidence interval
differences_fdr_corrected_table['P-val'] = pvals_2samp

In [38]:
def bold_significant_ps(x):
    return 'font-weight: bold' if x <= 0.05 else ''

In [39]:
differences_fdr_corrected_table.style \
.format(precision=6, thousands=',', decimal='.') \
.format_index(str.upper, axis=1) \
.applymap(bold_significant_ps, subset=['FDR P-val', 'P-val']) \
.hide()

LABEL ID,REGION NAME,Δ,FDR P-VAL,95% CI,P-VAL
14,Left Cerebrum.Temporal Lobe.Inferior Temporal Gyrus,0.018166,0.030863,0.009193,0.000561
20,Right Cerebrum.Temporal Lobe.Inferior Temporal Gyrus,0.012807,0.036174,0.005254,0.002960
24,Left Cerebrum.Temporal Lobe.Middle Temporal Gyrus,0.013364,0.035228,0.005689,0.002317
25,Left Cerebrum.Limbic Lobe.Uncus,0.010265,0.078200,0.002480,0.015418
27,Right Cerebrum.Limbic Lobe.Uncus,0.009825,0.079824,0.002187,0.017537
28,Right Cerebrum.Temporal Lobe.Middle Temporal Gyrus,0.011272,0.034655,0.005189,0.001319
39,Left Cerebrum.Temporal Lobe.Superior Temporal Gyrus,0.008758,0.079824,0.001844,0.019003
42,Right Cerebrum.Temporal Lobe.Superior Temporal Gyrus,0.006209,0.123676,0.000318,0.041600
51,Left Cerebrum.Temporal Lobe.Sub-Gyral,0.013137,0.035228,0.005516,0.002562
52,Right Cerebrum.Temporal Lobe.Sub-Gyral,0.012280,0.047984,0.004469,0.005235


In [40]:
stripped_Tal_results = differences_fdr_corrected_table.drop(differences_fdr_corrected_table[differences_fdr_corrected_table['FDR P-val'] > 0.05].index)

In [41]:
stripped_Tal_results.style \
.format(precision=4, thousands=',', decimal='.') \
.format_index(str.upper, axis=1) \
.hide()

LABEL ID,REGION NAME,Δ,FDR P-VAL,95% CI,P-VAL
14,Left Cerebrum.Temporal Lobe.Inferior Temporal Gyrus,0.0182,0.0309,0.0092,0.0006
20,Right Cerebrum.Temporal Lobe.Inferior Temporal Gyrus,0.0128,0.0362,0.0053,0.0030
24,Left Cerebrum.Temporal Lobe.Middle Temporal Gyrus,0.0134,0.0352,0.0057,0.0023
28,Right Cerebrum.Temporal Lobe.Middle Temporal Gyrus,0.0113,0.0347,0.0052,0.0013
51,Left Cerebrum.Temporal Lobe.Sub-Gyral,0.0131,0.0352,0.0055,0.0026
52,Right Cerebrum.Temporal Lobe.Sub-Gyral,0.0123,0.0480,0.0045,0.0052
134,Right Cerebrum.Limbic Lobe.Parahippocampal Gyrus,0.0114,0.0367,0.0046,0.0033
253,Right Cerebrum.Occipital Lobe.Lingual Gyrus,0.0169,0.0316,0.0082,0.0009
308,Left Cerebrum.Occipital Lobe.Middle Occipital Gyrus,0.0160,0.0347,0.0071,0.0018
486,Right Cerebrum.Occipital Lobe.Cuneus,0.0167,0.0239,0.0091,0.0002


In [42]:
import matplotlib.pyplot as plt
import dataframe_image as dfi
import latexcodec as lc

In [43]:
dfi.export(stripped_Tal_results.style \
.format(precision=4, thousands=',', decimal='.') \
.format_index(str.upper, axis=1) \
.hide(),'/home/corey/thesis-EP/EP_scripts/analysis/plots/Talairach_stripped.png',table_conversion='matplotlib')

/home/corey/anaconda3/envs/thesis-env/lib/python3.11/site-packages/dataframe_image/_pandas_accessor.py:69: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  html = '<div>' + obj.render() + '</div>'
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Font family 'Helvetica' not found.
findfont: Fo

In [ ]:
dfi.export(differences_fdr_corrected_table,'/home/corey/thesis-EP/EP_scripts/analysis/plots/Full_Talairach_WMdiffs.png',table_conversion='matplotlib',max_rows=-1)

In [45]:
#Create similar metrics controlling for age and sex... (simple multiple regression)
subject_wise_data_frame = pd.DataFrame(columns=['Subject ID','Age','Sex','Phenotype Description'])

In [47]:
fullSubjectInfo = pd.read_csv(open('/media/corey/4TB-WDBlue/data-thesis/subject_info.csv'),delim_whitespace=True)

In [49]:
subjectInfo_forRegression = pd.read_csv(open('/media/corey/4TB-WDBlue/data-thesis/subject_info.csv'),delim_whitespace=True, usecols=(4,6,7,11))

In [51]:
FA_forReg = []

for n in sub_names:
    path_to_data = subjectDir + n + DirExtName
    for r in white_matter_indices:
        fa_filename = path_to_data + '/av_%s_vol.nii.gz' %r
        if os.path.isfile(fa_filename) == False:
            continue
        fa_file = nib.load(fa_filename)
        fa_data = fa_file.get_fdata()
        FA_val = np.max(fa_data)
        FA_forReg.append([n, r, FA_val])

In [52]:
longlist_sub_names = [i[0] for i in FA_forReg]
unique_ll_sub_names = np.unique(longlist_sub_names)

In [53]:
subjectInfo_forRegression.head()

,src_subject_id,interview_age,sex,phenotype_description
0,"Subject,ID,how,it's,defined,in,lab/project","Age,in,months,at,the,time,of,the,interview/tes...","Sex,of,subject,at,birth","Description,of,the,phenotype,for,the,subject"
1,1048,290,M,"Non-affective,psychosis"
2,1050,347,F,"Non-affective,psychosis"
3,1051,219,M,"Non-affective,psychosis"
4,1052,249,M,"In,good,health"


In [ ]:
surrogate_df = pd.DataFrame(columns=['src_subject_id'])
surrogate_df['src_subject_id'] = unique_ll_sub_names
m = -1
c = 0
for n_ind, n in enumerate(unique_ll_sub_names):
    col = 0
    m += 1
    for t_ind, t in enumerate(stripped_wm_list):
        fa_val = FA_forReg[c][2]
        surrogate_df.at[m,col] = fa_val
        c +=1
        col += 1

In [55]:
#Concatenate subject info an surrogate dataframe containing FA values
test_merge = pd.merge(subjectInfo_forRegression, surrogate_df)


In [56]:
test_merge.sort_values(by=['src_subject_id']).head()

,src_subject_id,interview_age,sex,phenotype_description,0,1,2,3,4,5,...,100,101,102,103,104,105,106,107,108,109
116,1006,265,M,"Non-affective,psychosis",0.144902,0.111292,0.193110,0.131373,0.141319,0.155250,...,0.221461,0.195278,0.387719,0.461884,0.181751,0.185253,0.252555,0.325798,0.394071,0.137922
117,1009,277,F,"Non-affective,psychosis",0.131358,0.107020,0.193689,0.154669,0.157961,0.176701,...,0.283958,0.275045,0.189869,0.360076,0.392219,0.142630,0.195294,0.290753,0.318661,0.358518
118,1010,310,M,"In,good,health",0.310355,0.118516,0.130267,0.225596,0.131947,0.167088,...,0.178724,0.270244,0.220861,0.211816,0.283114,0.404297,0.162255,0.181597,0.272949,0.322552
119,1012,247,F,"Non-affective,psychosis",0.297324,0.248819,0.132324,0.119813,0.196316,0.092162,...,0.096703,0.185820,0.168841,0.158807,0.182523,0.425359,0.371815,0.185522,0.152220,0.323690
120,1013,358,F,"Non-affective,psychosis",0.342133,0.432541,0.425243,0.231076,0.159106,0.238536,...,0.300322,0.251386,0.371983,0.177747,0.285928,0.257125,0.357188,0.424039,0.190834,0.150150


In [57]:
no_na_test_merge = test_merge.dropna()

In [58]:
no_aff_subs_table = no_na_test_merge.drop(no_na_test_merge[no_na_test_merge.phenotype_description == 'Affective,psychosis'].index)

In [59]:
no_aff_subs_table['interview_age'] = no_aff_subs_table['interview_age'].astype(int)

In [60]:
sex_dict = {'M': 0, 'F': 1}
diagnosis_dict = {'In,good,health': 0, 'Non-affective,psychosis': 1}
no_aff_subs_table.sex = [sex_dict[item] for item in no_aff_subs_table.sex]
no_aff_subs_table.phenotype_description = [diagnosis_dict[item] for item in no_aff_subs_table.phenotype_description]

In [61]:
no_aff_subs_table

,src_subject_id,interview_age,sex,phenotype_description,0,1,2,3,4,5,...,100,101,102,103,104,105,106,107,108,109
0,1048,290,0,1,0.275925,0.277627,0.267198,0.135907,0.243185,0.199395,...,0.191236,0.166834,0.393583,0.367356,0.424888,0.292862,0.252245,0.463390,0.391795,0.187737
1,1051,219,0,1,0.187316,0.253449,0.280514,0.286051,0.171136,0.248535,...,0.184784,0.227177,0.221270,0.391035,0.367371,0.421237,0.253682,0.291496,0.493483,0.486945
2,1052,249,0,0,0.172712,0.169447,0.282790,0.274291,0.302312,0.274426,...,0.082693,0.162093,0.222747,0.183565,0.371942,0.365809,0.551605,0.239760,0.339987,0.495522
3,1053,290,0,1,0.389367,0.111405,0.166921,0.271513,0.287414,0.258832,...,0.230195,0.135464,0.172050,0.144524,0.118966,0.291119,0.246964,0.373540,0.227506,0.179187
4,1056,250,0,1,0.341501,0.246511,0.075666,0.087986,0.195693,0.227332,...,0.230164,0.281314,0.072008,0.100903,0.220967,0.187046,0.363913,0.351905,0.433799,0.282145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,1041,225,0,1,0.191929,0.242046,0.198180,0.391902,0.401072,0.285619,...,0.338264,0.404439,0.331296,0.291944,0.469570,0.393510,0.190402,0.158768,0.302448,0.326834
144,1043,283,0,1,0.238136,0.337630,0.223677,0.174235,0.364674,0.386710,...,0.366284,0.349095,0.509270,0.185589,0.223024,0.479079,0.347084,0.083111,0.141364,0.221629
145,1044,269,0,1,0.261941,0.277923,0.279498,0.237920,0.207771,0.391534,...,0.388139,0.349642,0.464712,0.251462,0.215748,0.446209,0.415530,0.158766,0.133070,0.248483
146,1045,284,0,1,0.261794,0.237357,0.280505,0.234063,0.157422,0.354527,...,0.166962,0.333286,0.313386,0.460161,0.233463,0.198014,0.421390,0.330669,0.121138,0.190691


In [62]:
no_aff_subs_table['age_sex_interaction'] = no_aff_subs_table['sex'] * no_aff_subs_table['interview_age']
no_aff_subs_table['age_diagnosis_interaction'] = no_aff_subs_table['phenotype_description'] * no_aff_subs_table['interview_age']
no_aff_subs_table['sex_diagnosis_interaction'] = no_aff_subs_table['sex'] * no_aff_subs_table['phenotype_description']
no_aff_subs_table['age_sex_diagnosis_interaction'] = no_aff_subs_table['sex'] * no_aff_subs_table['interview_age'] * no_aff_subs_table['phenotype_description']

In [63]:
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
import statsmodels as ssm

In [64]:
subs_with_EC = []

In [65]:
FA_table = no_aff_subs_table.iloc[:,4:114]

In [67]:
region_14 = no_aff_subs_table.iloc[:,13]

0      0.193719
1      0.306667
2      0.406872
3      0.416316
4      0.130754
         ...   
143    0.201620
144    0.384860
145    0.583112
146    0.563208
147    0.162928
Name: 9, Length: 123, dtype: float64

In [68]:
indep = no_aff_subs_table[['sex']]
X = ssm.tools.tools.add_constant(indep)
regression_phenotype_on_14 = ssm.regression.linear_model.OLS(region_14,X).fit()

In [69]:
regression_phenotype_on_14.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      9   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                 0.0007302
Date:                Sun, 10 Sep 2023   Prob (F-statistic):              0.978
Time:                        15:02:32   Log-Likelihood:                 84.486
No. Observations:                 123   AIC:                            -165.0
Df Residuals:                     121   BIC:                            -159.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2736      0.013     20.788      0.000       0.248       0.300
sex           -0.0007      0.024     -0.027      0.978      -0.049       0.048
==============================================================================
Omnibus:                        9.631   Durbin-Watson:                   1.145
Prob(Omnibus):                  0.008   Jarque-Bera (JB):                9.979
Skew:                           0.660   Prob(JB):                      0.00681
Kurtosis:                       2.550   Cond. No.                         2.43
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [70]:
models = []
preds = []
indep_list = ['sex','interview_age','phenotype_description']
indep = no_aff_subs_table['phenotype_description'] #no_aff_subs_table[['sex','interview_age','phenotype_description']]
X = ssm.tools.tools.add_constant(indep)
for x, c in enumerate(FA_table):
    if c in indep:
        continue
    dep = FA_table.iloc[:,x]
    model = ssm.regression.linear_model.OLS(dep,X).fit()
    pred = model.summary()
    models.append(models)
    preds.append(pred)

In [71]:
import scipy as sp

In [72]:
sw_pebs_mat = sp.io.loadmat('/media/corey/4TB-WDBlue/data-thesis/fMRI/bmrs/16July/sw_peb_bmrs.mat')

In [73]:
sw_pebs_mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'test_sw_bmr_1', 'test_sw_bmr_2'])

In [74]:
FA_table.head()

,0,1,2,3,4,5,6,7,8,9,...,100,101,102,103,104,105,106,107,108,109
0,0.275925,0.277627,0.267198,0.135907,0.243185,0.199395,0.389424,0.396461,0.279711,0.193719,...,0.191236,0.166834,0.393583,0.367356,0.424888,0.292862,0.252245,0.463390,0.391795,0.187737
1,0.187316,0.253449,0.280514,0.286051,0.171136,0.248535,0.243702,0.388695,0.413598,0.306667,...,0.184784,0.227177,0.221270,0.391035,0.367371,0.421237,0.253682,0.291496,0.493483,0.486945
2,0.172712,0.169447,0.282790,0.274291,0.302312,0.274426,0.228909,0.211397,0.402217,0.406872,...,0.082693,0.162093,0.222747,0.183565,0.371942,0.365809,0.551605,0.239760,0.339987,0.495522
3,0.389367,0.111405,0.166921,0.271513,0.287414,0.258832,0.206764,0.242971,0.202848,0.416316,...,0.230195,0.135464,0.172050,0.144524,0.118966,0.291119,0.246964,0.373540,0.227506,0.179187
4,0.341501,0.246511,0.075666,0.087986,0.195693,0.227332,0.173714,0.116951,0.179021,0.130754,...,0.230164,0.281314,0.072008,0.100903,0.220967,0.187046,0.363913,0.351905,0.433799,0.282145


In [75]:
FA_table.iloc[0,109]

0.1877371734259902

In [76]:
sub_list_ec = np.loadtxt('/home/corey/thesis-EP/EP_scripts/diagnostics/Logs/0Aff_29May_goodsubs_reduced.txt',dtype=str)
len(sub_list_ec)

96

In [77]:
only_ec_fa_table = no_aff_subs_table[no_aff_subs_table['src_subject_id'].isin(sub_list_ec)]
fa_table_onlyecsubs = only_ec_fa_table.iloc[:,4:114]

In [78]:
fa_table_onlyecsubs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 0 to 147
Columns: 110 entries, 0 to 109
dtypes: float64(110)
memory usage: 76.3 KB


In [79]:
list_88subs = list(set(sub_list).intersection(sub_list_ec))
len(list_88subs)
sorted_intersection_list = np.sort(list_88subs)

array(['1006', '1010', '1012', '1013', '1015', '1017', '1018', '1020',
       '1024', '1027', '1028', '1029', '1032', '1033', '1034', '1035',
       '1036', '1038', '1039', '1041', '1043', '1044', '1045', '1047',
       '1048', '1051', '1053', '1056', '1057', '1060', '1061', '1063',
       '1065', '1067', '1068', '1070', '1072', '1074', '1075', '1077',
       '1078', '1080', '1081', '1084', '1085', '1087', '1091', '1093',
       '1099', '1105', '2006', '2007', '2008', '2010', '2012', '2014',
       '2015', '2016', '2019', '2020', '2044', '2046', '2048', '3001',
       '3009', '3021', '3027', '3028', '3029', '3030', '3031', '3032',
       '3034', '3035', '4002', '4003', '4005', '4006', '4010', '4011',
       '4014', '4018', '4022', '4028', '4059', '4072', '4075', '4091'],
      dtype='<U4')

In [ ]:
non_members_peb = np.setdiff1d(sub_list_ec,sub_list)

In [81]:
binary_membership_pos_peb = np.zeros([96,1]) #Identfies the indices of the subjects in the list of subjects with peb data that are not in DTI dataset (should be 8)
for ind, sub in enumerate(sub_list_ec):
    if sub in non_members_peb:
        binary_membership_pos_peb[ind] = 1 #1s are in expected positions, 0s are in expected positions 

In [82]:
pos_in_5_sw_arr = np.zeros([88,21])
pos_out_5_sw_arr = np.zeros([88,21])
pos_in_4_sw_arr = np.zeros([88,21])
pos_out_4_sw_arr = np.zeros([88,21])

neg_in_5_sw_arr = np.zeros([88,21])
neg_out_5_sw_arr = np.zeros([88,21])
neg_in_4_sw_arr = np.zeros([88,21])
neg_out_4_sw_arr = np.zeros([88,21])
co = 0 #count offset
for n in np.arange(0,96,1):
    if binary_membership_pos_peb[n] == 1:
        co += 1
        continue
    for c in np.arange(0,21,1):
        pos_in_slow5 = 0
        pos_out_slow5 = 0
        pos_in_slow4 = 0
        pos_out_slow4 = 0

        neg_in_slow5 = 0
        neg_out_slow5 = 0
        neg_in_slow4 = 0
        neg_out_slow4 = 0
        for r in np.arange(0,21,1):
            if sw_pebs_mat['test_sw_bmr_1']['Ep'][n][0][0][0][0][c][r] > 0:
                pos_out_slow5 += sw_pebs_mat['test_sw_bmr_1']['Ep'][n][0][0][0][0][c][r]
            if sw_pebs_mat['test_sw_bmr_1']['Ep'][n][0][0][0][0][r][c] > 0:
                pos_in_slow5 += sw_pebs_mat['test_sw_bmr_1']['Ep'][n][0][0][0][0][r][c]
            if sw_pebs_mat['test_sw_bmr_2']['Ep'][n][0][0][0][0][c][r] > 0:
                pos_out_slow4 += sw_pebs_mat['test_sw_bmr_2']['Ep'][n][0][0][0][0][c][r]
            if sw_pebs_mat['test_sw_bmr_2']['Ep'][n][0][0][0][0][r][c] > 0:
                pos_in_slow4 += sw_pebs_mat['test_sw_bmr_2']['Ep'][n][0][0][0][0][r][c]

            if sw_pebs_mat['test_sw_bmr_1']['Ep'][n][0][0][0][0][c][r] < 0:
                neg_out_slow5 += sw_pebs_mat['test_sw_bmr_1']['Ep'][n][0][0][0][0][c][r]
            if sw_pebs_mat['test_sw_bmr_1']['Ep'][n][0][0][0][0][r][c] < 0:
                neg_in_slow5 += sw_pebs_mat['test_sw_bmr_1']['Ep'][n][0][0][0][0][r][c]
            if sw_pebs_mat['test_sw_bmr_2']['Ep'][n][0][0][0][0][c][r] < 0:
                neg_out_slow4 += sw_pebs_mat['test_sw_bmr_2']['Ep'][n][0][0][0][0][c][r]
            if sw_pebs_mat['test_sw_bmr_2']['Ep'][n][0][0][0][0][r][c] < 0:
                neg_in_slow4 += sw_pebs_mat['test_sw_bmr_2']['Ep'][n][0][0][0][0][r][c]


        pos_in_5_sw_arr[n-co,c] = pos_in_slow5
        pos_out_5_sw_arr[n-co,c] = pos_out_slow5
        pos_in_4_sw_arr[n-co,c] = pos_in_slow4
        pos_out_4_sw_arr[n-co,c] = pos_out_slow4

        neg_in_5_sw_arr[n-co,c] = neg_in_slow5
        neg_out_5_sw_arr[n-co,c] = neg_out_slow5
        neg_in_4_sw_arr[n-co,c] = neg_in_slow4
        neg_out_4_sw_arr[n-co,c] = neg_out_slow4

In [84]:
#ROIs in DCMs corresponding to labels of white matter regions in Talairach atlas

#PCC Composite of 662 and 664 [64, 65]
PCC_fa_vec = np.zeros([88,1])
for n in np.arange(0,88,1):
    PCC_fa_vec[n] = (fa_table_onlyecsubs.iloc[n,64] + fa_table_onlyecsubs.iloc[n,65]) / 2
#mPFC Composite of 230 and 231 [30, 31]
mPFC_fa_vec = np.zeros([88,1])
for n in np.arange(0,88,1):
    mPFC_fa_vec[n] = (fa_table_onlyecsubs.iloc[n,30] + fa_table_onlyecsubs.iloc[n,31]) / 2
#LlPar 965, RlPar 966 [100, 101]
LlPar_fa_vec = np.zeros([88,1])
for n in np.arange(0,88,1):
    LlPar_fa_vec[n] = fa_table_onlyecsubs.iloc[n,100]
RlPar_fa_vec = np.zeros([88,1])
for n in np.arange(0,88,1):
    RlPar_fa_vec[n] = fa_table_onlyecsubs.iloc[n,101]
#LiTem 14 RiTem 20 [0, 1]100
LiTem_fa_vec = np.zeros([88,1])
for n in np.arange(0,88,1):
    LiTem_fa_vec[n] = fa_table_onlyecsubs.iloc[n,14]
RiTem_fa_vec = np.zeros([88,1])
for n in np.arange(0,88,1):
    RiTem_fa_vec[n] = fa_table_onlyecsubs.iloc[n,20]
#dmPFC & RLaPFC + RLaPFC_SN share with mPFC?

#LsPar 1041 RsPar 1042 [104, 105]
LsPar_fa_vec = np.zeros([88,1])
for n in np.arange(0,88,1):
    LsPar_fa_vec[n] = fa_table_onlyecsubs.iloc[n,104]
RsPar_fa_vec = np.zeros([88,1])
for n in np.arange(0,88,1):
    RsPar_fa_vec[n] = fa_table_onlyecsubs.iloc[n,105]
#dACC Composite 371 & 374 (not perfect match) [48, 49]
dACC_fa_vec = np.zeros([88,1])
for n in np.arange(0,88,1):
    dACC_fa_vec[n] = (fa_table_onlyecsubs.iloc[n,48] + fa_table_onlyecsubs.iloc[n,49]) / 2
#L Insula 439 R Insula 441 [50, 51]
LIns_fa_vec = np.zeros([88,1])
for n in np.arange(0,88,1):
    LIns_fa_vec[n] = fa_table_onlyecsubs.iloc[n,50]
RIns_fa_vec = np.zeros([88,1])
for n in np.arange(0,88,1):
    RIns_fa_vec[n] = fa_table_onlyecsubs.iloc[n,51]
#LlPar_SN 929 RlPar_SN 930 [92, 93] (Angular Gyri)
LlPar_SN_fa_vec = np.zeros([88,1])
for n in np.arange(0,88,1):
    LlPar_SN_fa_vec[n] = fa_table_onlyecsubs.iloc[n,92]
RlPar_SN_fa_vec = np.zeros([88,1])
for n in np.arange(0,88,1):
    RlPar_SN_fa_vec[n] = fa_table_onlyecsubs.iloc[n,93]

In [85]:
#test_reg_PCC
indep_pcc = PCC_fa_vec
Xpcc = ssm.tools.tools.add_constant(indep_pcc)
fa_ec_regression_pcc = ssm.regression.linear_model.OLS(pos_in_5_sw_arr[:,0],Xpcc).fit()

indep_llpar = LlPar_fa_vec
Xllpar = ssm.tools.add_constant(indep_llpar)
fa_ec_regression_llpar = ssm.regression.linear_model.OLS(pos_out_5_sw_arr[:,2],Xllpar).fit()

indep_rlpar = RlPar_fa_vec
Xrlpar = ssm.tools.add_constant(indep_rlpar)
fa_ec_regression_rlpar = ssm.regression.linear_model.OLS(pos_out_5_sw_arr[:,3],Xrlpar).fit()

indep_litem = LiTem_fa_vec
Xlitem = ssm.tools.tools.add_constant(indep_litem)
fa_ec_regression_litem = ssm.regression.linear_model.OLS(pos_in_5_sw_arr[:,4],Xlitem).fit()

indep_ritem = RiTem_fa_vec
Xritem = ssm.tools.tools.add_constant(indep_ritem)
fa_ec_regression_ritem = ssm.regression.linear_model.OLS(pos_in_5_sw_arr[:,5],Xritem).fit()

indep_llparsn = LlPar_SN_fa_vec
Xllparsn = ssm.tools.tools.add_constant(indep_llparsn)
fa_ec_regression_llparsn = ssm.regression.linear_model.OLS(pos_out_5_sw_arr[:,20],Xllparsn).fit()

In [86]:
fa_ec_regression_ritem.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.011
Method:                 Least Squares   F-statistic:                   0.06074
Date:                Sun, 10 Sep 2023   Prob (F-statistic):              0.806
Time:                        15:02:51   Log-Likelihood:                -264.59
No. Observations:                  88   AIC:                             533.2
Df Residuals:                      86   BIC:                             538.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.3693      1.218      5.229      0.000       3.948       8.791
x1             1.0105      4.100      0.246      0.806      -7.140       9.162
==============================================================================
Omnibus:                       18.658   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               22.401
Skew:                           1.162   Prob(JB):                     1.37e-05
Kurtosis:                       3.840   Cond. No.                         8.34
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""